In [1]:
## Relevant modules.
import pandas as pd
import os

In [2]:
## Reading files.
file_2008 = 'data/data_outpatient_2008.csv'
file_2009 = 'data/data_outpatient_2009.csv'
file_2010 = 'data/data_outpatient_2010.csv'
file_2011 = 'data/data_outpatient_2011.csv'
file_2012 = 'data/data_outpatient_2012.csv'
file_costs = 'data/mean_outpatient_costs_usd.csv'
file_dep = 'data/data_departments.csv'
data_outpatient_2008 = pd.read_csv(file_2008, header=0)
data_outpatient_2009 = pd.read_csv(file_2009, header=0)
data_outpatient_2010 = pd.read_csv(file_2010, header=0)
data_outpatient_2011 = pd.read_csv(file_2011, header=0)
data_outpatient_2012 = pd.read_csv(file_2012, header=0)
mean_outpatient_costs_usd = pd.read_csv(file_costs, header=0)
data_dep = pd.read_csv(file_dep, header=0)
## Expected length when concatenating 2008-2012 files. 
exp_len = (len(data_outpatient_2008)+
           len(data_outpatient_2009)+
           len(data_outpatient_2010)+
           len(data_outpatient_2011)+
           len(data_outpatient_2012))
## Expected total visits when concatenating 2008-2012 files. 
exp_vis = (data_outpatient_2008['n_visits'].sum()+
           data_outpatient_2009['n_visits'].sum()+
           data_outpatient_2010['n_visits'].sum()+
           data_outpatient_2011['n_visits'].sum()+
           data_outpatient_2012['n_visits'].sum())
## Expected total surgeries when concatenating 2008-2012 files. 
exp_sur = (data_outpatient_2008['n_surgeries'].sum()+
           data_outpatient_2009['n_surgeries'].sum()+
           data_outpatient_2010['n_surgeries'].sum()+
           data_outpatient_2011['n_surgeries'].sum()+
           data_outpatient_2012['n_surgeries'].sum())
## Expected total patients when concatenating 2008-2012 files. 
exp_pat = (data_outpatient_2008['n_patients'].sum()+
           data_outpatient_2009['n_patients'].sum()+
           data_outpatient_2010['n_patients'].sum()+
           data_outpatient_2011['n_patients'].sum()+
           data_outpatient_2012['n_patients'].sum())
print('The expected length when concatenating 2008-2012 files is', exp_len, end='.\n')
print('The expected total visits is', exp_vis, end='.\n')
print('The expected total surgeries is', exp_sur, end='.\n')
print('The expected total patients is', exp_pat, end='.')


The expected length when concatenating 2008-2012 files is 9420.
The expected total visits is 1044072.
The expected total surgeries is 108118.
The expected total patients is 870724.

In [3]:
## Concatenating 2008-2012 files.
concat_data = pd.concat([data_outpatient_2008,\
                         data_outpatient_2009,\
                         data_outpatient_2010,\
                         data_outpatient_2011,\
                         data_outpatient_2012],
                         ignore_index=True)

print(concat_data.head(), '\n')
print(concat_data.tail(), '\n')
print(concat_data.info(), '\n')
print('Length as expected?', len(concat_data) == exp_len, end='.') 

   spec_code                       spec_es  year  doc_code  n_visits  \
0         22  anestesiologia y reanimacion  2008  16449291        59   
1         22  anestesiologia y reanimacion  2008  31865019        85   
2         22  anestesiologia y reanimacion  2008   8682278         6   
3        120                   cardiologia  2008  19188542        24   
4        120                   cardiologia  2008  70115939       429   

   n_surgeries  n_patients  
0            1          67  
1            2          89  
2            4           6  
3            3          24  
4          104         430   

      spec_code                   spec_es  year  doc_code  n_visits  \
9415        775       urologia pediatrica  2012  79939705        48   
9416        775       urologia pediatrica  2012  79557883        82   
9417        992  cirugia de hombro y codo  2012  80418242       105   
9418        999          medicina general  2012  41797433        27   
9419        999          medicina ge

In [4]:
## Grouping by spec_code, spec_es, year and doc_code.
concat_data_unique = concat_data.groupby(['spec_code',\
                                          'spec_es',\
                                          'year',\
                                          'doc_code'],\
                                           as_index=False, sort=False).sum()

tot_vis = concat_data_unique['n_visits'].sum()
tot_sur = concat_data_unique['n_surgeries'].sum()
tot_pat = concat_data_unique['n_patients'].sum()

print(concat_data_unique.head(), '\n')
print(concat_data_unique.tail(), '\n')
print(concat_data_unique.info(), '\n')
print('Total visits is', tot_vis, end='.'); print(' As expected?', tot_vis==exp_vis, end='.\n')
print('Total surgeries is', tot_sur, end='.'); print(' As expected?', tot_sur==exp_sur, end='.\n')
print('Total patients is', tot_pat, end='.'); print(' As expected?', tot_pat==exp_pat, end='.')

   spec_code                       spec_es  year  doc_code  n_visits  \
0         22  anestesiologia y reanimacion  2008  16449291        59   
1         22  anestesiologia y reanimacion  2008  31865019        85   
2         22  anestesiologia y reanimacion  2008   8682278         6   
3        120                   cardiologia  2008  19188542        24   
4        120                   cardiologia  2008  70115939       429   

   n_surgeries  n_patients  
0            1          67  
1            2          89  
2            4           6  
3            3          24  
4          104         430   

      spec_code                   spec_es  year  doc_code  n_visits  \
8299        775       urologia pediatrica  2012  79939705        48   
8300        775       urologia pediatrica  2012  79557883        82   
8301        992  cirugia de hombro y codo  2012  80418242       105   
8302        999          medicina general  2012  41797433        27   
8303        999          medicina ge

In [5]:
## Putting everything together, a.k.a. merging.
panel_temp = pd.merge(concat_data_unique, mean_outpatient_costs_usd,\
                      on=['spec_code','spec_es','year'], how='left')
panel = pd.merge(panel_temp, data_dep,\
                 on=['spec_code','spec_es','year','doc_code'], how='left')

print(panel.head(), '\n')
print(panel.tail(), '\n')
print(panel.info())

   spec_code                       spec_es  year  doc_code  n_visits  \
0         22  anestesiologia y reanimacion  2008  16449291        59   
1         22  anestesiologia y reanimacion  2008  31865019        85   
2         22  anestesiologia y reanimacion  2008   8682278         6   
3        120                   cardiologia  2008  19188542        24   
4        120                   cardiologia  2008  70115939       429   

   n_surgeries  n_patients                           spec_en  c_surgery  \
0            1          67  anesthesiology and resuscitation     205.85   
1            2          89  anesthesiology and resuscitation     205.85   
2            4           6  anesthesiology and resuscitation     205.85   
3            3          24                        cardiology     209.02   
4          104         430                        cardiology     209.02   

   c_visit  dep_code       department  
0    21.69        76  valle del cauca  
1    21.69        76  valle del cauc

In [6]:
## Making some rearrangement.
col_order = ['doc_code','spec_code','spec_es','spec_en',\
             'year','n_visits','n_surgeries','n_patients',\
             'c_visit','c_surgery','dep_code','department']
panel = panel[col_order]
print(panel.head(), '\n')
print(panel.tail(), '\n')
print(panel.info())

   doc_code  spec_code                       spec_es  \
0  16449291         22  anestesiologia y reanimacion   
1  31865019         22  anestesiologia y reanimacion   
2   8682278         22  anestesiologia y reanimacion   
3  19188542        120                   cardiologia   
4  70115939        120                   cardiologia   

                            spec_en  year  n_visits  n_surgeries  n_patients  \
0  anesthesiology and resuscitation  2008        59            1          67   
1  anesthesiology and resuscitation  2008        85            2          89   
2  anesthesiology and resuscitation  2008         6            4           6   
3                        cardiology  2008        24            3          24   
4                        cardiology  2008       429          104         430   

   c_visit  c_surgery  dep_code       department  
0    21.69     205.85        76  valle del cauca  
1    21.69     205.85        76  valle del cauca  
2    21.69     205.85        

In [7]:
## Issue 01: errors with spec_es or spec_code.
filter_01 = panel['spec_en'].isnull()
print(panel[filter_01])

      doc_code  spec_code                                   spec_es spec_en  \
792   71578770        342                 ginecologia laparoscopica     NaN   
2370  71578770        342                 ginecologia laparoscopica     NaN   
3986  71578770        342                 ginecologia laparoscopica     NaN   
4840  80418242        992                 ortopedia y traumatologia     NaN   
5656  71578770        342                 ginecologia laparoscopica     NaN   
6744  73146094        139  ciurgia de la mama y tum. de tejidos bla     NaN   
7393  70563663        342                 ginecologia laparoscopica     NaN   
7394  71578770        342                 ginecologia laparoscopica     NaN   
7395  79148518        342                 ginecologia laparoscopica     NaN   
7986  12547744        516                 ortopedia y traumatologia     NaN   

      year  n_visits  n_surgeries  n_patients  c_visit  c_surgery  dep_code  \
792   2008        84            7          74      

In [8]:
## Issue 02: no information about costs.
filter_02 = panel['c_visit'].isnull()
filter_03 = panel['c_surgery'].isnull()
print(panel[~filter_01 & (filter_02 | filter_03)])

      doc_code  spec_code                       spec_es  \
3188  16449291         22  anestesiologia y reanimacion   
3189   8682278         22  anestesiologia y reanimacion   
4847  15349090         22  anestesiologia y reanimacion   
4848  31988075         22  anestesiologia y reanimacion   
4849  39779019         22  anestesiologia y reanimacion   
4850   8693061         22  anestesiologia y reanimacion   

                               spec_en  year  n_visits  n_surgeries  \
3188  anesthesiology and resuscitation  2010        84            1   
3189  anesthesiology and resuscitation  2010        10            8   
4847  anesthesiology and resuscitation  2011        22            3   
4848  anesthesiology and resuscitation  2011        37            1   
4849  anesthesiology and resuscitation  2011         5            1   
4850  anesthesiology and resuscitation  2011         6            3   

      n_patients  c_visit  c_surgery  dep_code       department  
3188          90      

In [9]:
## Saving data_panel without issue 01: spec_es or spec_code errors.
data_panel = panel[~filter_01]
data_panel.reset_index(drop=True, inplace=True)
print(data_panel.info(),'\n')
## Saving data_panel as csv to data folder.
if os.path.exists('data/data_panel.csv'):
    print('data/data_panel.csv already exists.')
elif os.path.exists('data'):
    file_out = 'data/data_panel.csv'
    data_panel.to_csv(path_or_buf=file_out, index=False)
    print('data/ already exists.')
    print('data/data_panel.csv has been created.')
else:
    os.mkdir('data')
    file_out = 'data/data_panel.csv'
    data_panel.to_csv(path_or_buf=file_out, index=False)
    print('data/data_panel.csv has been created.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8294 entries, 0 to 8293
Data columns (total 12 columns):
doc_code       8294 non-null int64
spec_code      8294 non-null int64
spec_es        8294 non-null object
spec_en        8294 non-null object
year           8294 non-null int64
n_visits       8294 non-null int64
n_surgeries    8294 non-null int64
n_patients     8294 non-null int64
c_visit        8288 non-null float64
c_surgery      8292 non-null float64
dep_code       8294 non-null int64
department     8294 non-null object
dtypes: float64(2), int64(7), object(3)
memory usage: 777.6+ KB
None 

data/ already exists.
data/data_panel.csv has been created.
